In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
! pip install gdown

#AHS_clear Dataset
!gdown --id 1ejVLlApcdG77h-zIL7NPbnaiPDsUooLV
!gdown --id 1im9nJFcwz--jODqsKYD5qNqbr7st0UhC
#AHS Dataset
!gdown --id 122xV9jV2FCb9-cNiQPHG7dLoCkk2IsfF 
#download vocab file
!gdown --id 16Tiw2mAtHvdh3OZhffq-heG_j8d5ADZK
#download word embadding model
!gdown --id 1Tlfc3caP6wAf7e_9URZJ5A2Mvt7Uc1U9
!gdown --id 1uqJ_YkvBKjCUcKeawiOkvfgGPAydUWXe

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:130: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From (uriginal): https://drive.google.com/uc?id=1ejVLlApcdG77h-zIL7NPbnaiPDsUooLV
From (redirected): https://drive.google.com/uc?id=1ejVLlApcdG77h-zIL7NPbnaiPDsUooLV&confirm=t&uuid=e5f4393f-bf47-46d7-b53b-e1d90fdb56b2
To: /kaggle/working/AHS_clear.csv
100%|████████████████████████████████████████| 186M/186M [00:02<00:00, 88.7MB/s]
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:130: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From (uriginal): https://drive.google.com/uc?id=1im9nJFcwz--jODqsKYD5qNqbr7st0UhC
From (redirected): https://drive.google.com/uc?id=1im9nJFcwz--jODqsKYD5qNqbr7st0UhC&confirm=t&uuid=50b56f26

In [3]:
'''
# To Restore Model
import gdown
url='https://drive.google.com/drive/folders/1cEbqKr68i3x4-llV-oE4U5m-FHoncmvf'
gdown.download_folder(url, quiet=True)
'''

"\n# To Restore Model\nimport gdown\nurl='https://drive.google.com/drive/folders/1cEbqKr68i3x4-llV-oE4U5m-FHoncmvf'\ngdown.download_folder(url, quiet=True)\n"

In [4]:
!pip install arabic-reshaper

In [5]:
from __future__ import absolute_import, division, print_function
# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf
#tf.enable_eager_execution()
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from bidi.algorithm import get_display
import arabic_reshaper
import unicodedata
import re
import numpy as np
import os
import time
import math 


import tensorflow_addons as tfa
import matplotlib.ticker as ticker
import io


In [6]:
# Path for the dataset file
path_to_file =r'/kaggle/working/AHS_pandas.txt'

In [7]:
class ATSDataset:
    def __init__(self, problem_type='ATS'):
        self.problem_type = 'ATS'
        self.inp_lang_tokenizer = None
        self.targ_lang_tokenizer = None


    def unicode_to_ascii(self, s):
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

    ## Step 1 and Step 2 
    def preprocess_sentence(self, w):
        '''
    Args:
        w : A single word
    Returns:
        w : Single normalize word 
    
    Convert Unicode to ASCII
    Creating a space between a word and the punctuation following it
    eg: "he is a boy." => "he is a boy ." 
    Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    
    Replacing everything with space except (a-z, A-Z, ا-ي ".", "?", "!", ",")
    
    Adding a start and an end token to the sentence
    
    '''
        #w = self.unicode_to_ascii(w.strip())
    
        w = re.sub(r"([?.!,¿])", r" \1 ", w)
        w = re.sub(r'[" "]+', " ", w)
    
        w = re.sub(r"[^a-zA-Z؀-ۿ?.!,¿]+", " ", w)
        w = w.rstrip().strip()
    

        # adding a start and an end token to the sentence
        # so that the model know when to start and stop predicting.
        w = '<start> %s <end>'%w
        return w

    def create_dataset(self, path, num_examples):
        '''
        Args:
        path: Path of the dataset file
        num_examples: Threshold to read a range from dataset
        Returns:
        word_pairs : All readed words from dataset as a pairs 
        
        1. Remove the accents
        2. Clean the sentences
        3. Return word pairs in the format: [Article, Summary]
        '''
        lines = open(path, encoding='utf-8-sig').read().strip().split('\n')
        word_pairs = [[self.preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
        print(len(lines))
        print(len(lines[:num_examples]))
        return word_pairs

    # Step 3 and Step 4
    def tokenize(self, lang):
        # lang = list of sentences in a language

        # print(len(lang), "example sentence: {}".format(lang[0]))
        lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>')
        lang_tokenizer.fit_on_texts(lang)

        ## tf.keras.preprocessing.text.Tokenizer.texts_to_sequences converts string (w1, w2, w3, ......, wn) 
        ## to a list of correspoding integer ids of words (id_w1, id_w2, id_w3, ...., id_wn)
        tensor = lang_tokenizer.texts_to_sequences(lang) 

        ## tf.keras.preprocessing.sequence.pad_sequences takes argument a list of integer id sequences 
        ## and pads the sequences to match the longest sequences in the given input
        tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

        return tensor, lang_tokenizer

    def load_dataset(self, path, num_examples=None):
        # creating cleaned input, output pairs
        inp_lang,targ_lang = zip(*self.create_dataset(path, num_examples))

        input_tensor, inp_lang_tokenizer = self.tokenize(inp_lang)
        target_tensor, targ_lang_tokenizer = self.tokenize(targ_lang)

        return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

    def call(self, num_examples, BUFFER_SIZE, BATCH_SIZE):
        file_path = path_to_file
        input_tensor, target_tensor, self.inp_lang_tokenizer, self.targ_lang_tokenizer = self.load_dataset(file_path, num_examples)

        input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

        train_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train))
        train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

        val_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val))
        val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

        return train_dataset, val_dataset, self.inp_lang_tokenizer, self.targ_lang_tokenizer

In [8]:
BUFFER_SIZE = 32000
BATCH_SIZE = 64
# Let's limit the #training examples for faster training
num_examples = 100000

dataset_creator = ATSDataset('ATS')
train_dataset, val_dataset, inp_lang, targ_lang = dataset_creator.call(num_examples, BUFFER_SIZE, BATCH_SIZE)

589672
100000


In [9]:
example_input_batch, example_target_batch = next(iter(train_dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 284]), TensorShape([64, 16]))

In [10]:
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1
max_length_input = example_input_batch.shape[1]
max_length_output = example_target_batch.shape[1]

embedding_dim = 256
units = 1024
steps_per_epoch = num_examples//BATCH_SIZE

In [11]:
print("max_length_input, max_length_output, vocab_size_input, vocab_size_output")
max_length_input, max_length_output, vocab_inp_size, vocab_tar_size

max_length_input, max_length_output, vocab_size_input, vocab_size_output


(284, 16, 179582, 26567)

In [12]:
##### 

class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    ##-------- LSTM layer in Encoder ------- ##
        self.lstm_layer = tf.keras.layers.LSTM(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')



    def call(self, x, hidden):
        x = self.embedding(x)
        output, h, c = self.lstm_layer(x, initial_state = hidden)
        return output, h, c

    def initialize_hidden_state(self):
        return [tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units))]

In [13]:
# Test Encoder Stack

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)


# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_h, sample_c = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder h vecotr shape: (batch size, units) {}'.format(sample_h.shape))
print ('Encoder c vector shape: (batch size, units) {}'.format(sample_c.shape))

Encoder output shape: (batch size, sequence length, units) (64, 284, 1024)
Encoder h vecotr shape: (batch size, units) (64, 1024)
Encoder c vector shape: (batch size, units) (64, 1024)


In [14]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, attention_type='luong'):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.attention_type = attention_type

    # Embedding Layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    #Final Dense layer on which softmax will be applied
        self.fc = tf.keras.layers.Dense(vocab_size)

    # Define the fundamental cell for decoder recurrent structure
        self.decoder_rnn_cell = tf.keras.layers.LSTMCell(self.dec_units)
    
    # Sampler
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()

    # Create attention mechanism with memory = None
        self.attention_mechanism = self.build_attention_mechanism(self.dec_units, 
                                                              None, self.batch_sz*[max_length_input], self.attention_type)

    # Wrap attention mechanism with the fundamental rnn cell of decoder
        self.rnn_cell = self.build_rnn_cell(batch_sz)

    # Define the decoder with respect to fundamental rnn cell
        self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler=self.sampler, output_layer=self.fc)


    def build_rnn_cell(self, batch_sz):
        rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnn_cell, 
                                  self.attention_mechanism, attention_layer_size=self.dec_units)
        return rnn_cell

    def build_attention_mechanism(self, dec_units, memory, memory_sequence_length, attention_type='luong'):
    # ------------- #
    # typ: Which sort of attention (Bahdanau, Luong)
    # dec_units: final dimension of attention outputs 
    # memory: encoder hidden states of shape (batch_size, max_length_input, enc_units)
    # memory_sequence_length: 1d array of shape (batch_size) with every element set to max_length_input (for masking purpose)

        if(attention_type=='bahdanau'):
            return tfa.seq2seq.BahdanauAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)
        else:
            return tfa.seq2seq.LuongAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)

    def build_initial_state(self, batch_sz, encoder_state, Dtype):
        decoder_initial_state = self.rnn_cell.get_initial_state(batch_size=batch_sz, dtype=Dtype)
        decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)
        return decoder_initial_state


    def call(self, inputs, initial_state):
        x = self.embedding(inputs)
        outputs, _, _ = self.decoder(x, initial_state=initial_state, sequence_length=self.batch_sz*[max_length_output-1])
        return outputs

In [15]:
# Test decoder stack

decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 'luong')
sample_x = tf.random.uniform((BATCH_SIZE, max_length_output))
decoder.attention_mechanism.setup_memory(sample_output)
initial_state = decoder.build_initial_state(BATCH_SIZE, [sample_h, sample_c], tf.float32)


sample_decoder_outputs = decoder(sample_x, initial_state)

print("Decoder Outputs Shape: ", sample_decoder_outputs.rnn_output.shape)

Decoder Outputs Shape:  (64, 15, 26567)


In [16]:
optimizer = tf.keras.optimizers.Adam()

def loss_function(real, pred):
  # real shape = (BATCH_SIZE, max_length_output)
  # pred shape = (BATCH_SIZE, max_length_output, tar_vocab_size )
    cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    loss = cross_entropy(y_true=real, y_pred=pred)
    mask = tf.logical_not(tf.math.equal(real,0))   #output 0 for y=0 else output 1
    mask = tf.cast(mask, dtype=loss.dtype)  
    loss = mask* loss
    loss = tf.reduce_mean(loss)
    return loss

In [17]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [18]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_h, enc_c = encoder(inp, enc_hidden)

        dec_input = targ[ : , :-1 ] # Ignore <end> token
        real = targ[ : , 1: ]         # ignore <start> token

    # Set the AttentionMechanism object with encoder_outputs
        decoder.attention_mechanism.setup_memory(enc_output)

    # Create AttentionWrapperState as initial_state for decoder
        decoder_initial_state = decoder.build_initial_state(BATCH_SIZE, [enc_h, enc_c], tf.float32)
        pred = decoder(dec_input, decoder_initial_state)
        logits = pred.rnn_output
        loss = loss_function(real, logits)

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return loss

In [19]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
  # print(enc_hidden[0].shape, enc_hidden[1].shape)

    for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.8973
Epoch 1 Batch 100 Loss 1.8488
Epoch 1 Batch 200 Loss 1.9416
Epoch 1 Batch 300 Loss 1.7970
Epoch 1 Batch 400 Loss 1.8095
Epoch 1 Batch 500 Loss 1.6768
Epoch 1 Batch 600 Loss 1.6252
Epoch 1 Batch 700 Loss 1.5758
Epoch 1 Batch 800 Loss 1.8248
Epoch 1 Batch 900 Loss 1.5714
Epoch 1 Batch 1000 Loss 1.7086
Epoch 1 Batch 1100 Loss 1.4937
Epoch 1 Batch 1200 Loss 1.6901
Epoch 1 Loss 1.3803
Time taken for 1 epoch 609.6581337451935 sec

Epoch 2 Batch 0 Loss 1.4772
Epoch 2 Batch 100 Loss 1.4819
Epoch 2 Batch 200 Loss 1.4771
Epoch 2 Batch 300 Loss 1.3902
Epoch 2 Batch 400 Loss 1.5351
Epoch 2 Batch 500 Loss 1.2664
Epoch 2 Batch 600 Loss 1.2992
Epoch 2 Batch 700 Loss 1.3993
Epoch 2 Batch 800 Loss 1.4016
Epoch 2 Batch 900 Loss 1.2949
Epoch 2 Batch 1000 Loss 1.3751
Epoch 2 Batch 1100 Loss 1.4495
Epoch 2 Batch 1200 Loss 1.3433
Epoch 2 Loss 1.1423
Time taken for 1 epoch 600.9999649524689 sec

Epoch 3 Batch 0 Loss 1.2010
Epoch 3 Batch 100 Loss 1.2918
Epoch 3 Batch 200 Loss 1.353

In [20]:
def evaluate_sentence(sentence):
    sentence = dataset_creator.preprocess_sentence(sentence)
    
    ##This is a proplem##############
    inputs=''
    try:
        inputs = [inp_lang.word_index[i] for i in sentence.split()]
    except:
        pass
    ############################
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_length_input,
                                                          padding='post')
    inputs = tf.convert_to_tensor(inputs)
    inference_batch_size = inputs.shape[0]
    result = ''

    enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
    enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)

    dec_h = enc_h
    dec_c = enc_c
    
    start_tokens = tf.fill([inference_batch_size], targ_lang.word_index['<start>'])
    end_token = targ_lang.word_index['<end>']

    greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()
    
  # Instantiate BasicDecoder object
    decoder_instance = tfa.seq2seq.BasicDecoder(cell=decoder.rnn_cell, sampler=greedy_sampler, output_layer=decoder.fc)
  # Setup Memory in decoder stack
    decoder.attention_mechanism.setup_memory(enc_out)

  # set decoder_initial_state
    decoder_initial_state = decoder.build_initial_state(inference_batch_size, [enc_h, enc_c], tf.float32)
    

  ### Since the BasicDecoder wraps around Decoder's rnn cell only, you have to ensure that the inputs to BasicDecoder 
  ### decoding step is output of embedding layer. tfa.seq2seq.GreedyEmbeddingSampler() takes care of this. 
  ### You only need to get the weights of embedding layer, which can be done by decoder.embedding.variables[0] and pass this callabble to BasicDecoder's call() function

    decoder_embedding_matrix = decoder.embedding.variables[0]

    outputs, _, _ = decoder_instance(decoder_embedding_matrix, start_tokens = start_tokens, end_token= end_token, initial_state=decoder_initial_state)
    
    return outputs.sample_id.numpy()

def Summary(sentence):
    result = evaluate_sentence(sentence)
    #print(result)
    result = targ_lang.sequences_to_texts(result)
    return result
    #print('Input: %s' % (sentence))
    #print('Predicted summarization: {}'.format(result))

In [21]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [22]:
import pandas as pd
df = pd.read_csv('/kaggle/working/AHS_clear.csv')[:num_examples]
df = df.filter(['Text', 'Summary'])

In [23]:
Summary(df['Text'][90000])

['مدينه الناظور <end>']

In [24]:
df['Summary'][90000]

'مدينه الضمير'

In [25]:
Text_list,Predicted_list,Summary_list=[],[],[]
for text,summary in zip(df['Text'],df['Summary']):
    Text_list.append(text)
    Predicted_list.append(Summary(text))
    Summary_list.append(summary)

In [26]:
dic={'Text':Text_list,'Predicted Summary':Predicted_list,'Real Summary':Summary_list}
ds=pd.DataFrame(dic)
ds.to_csv('out.csv', index=False)

In [27]:
def beam_evaluate_sentence(sentence, beam_width=3):
    sentence = dataset_creator.preprocess_sentence(sentence)

    ##This is a proplem##############
    inputs=''
    try:
        inputs = [inp_lang.word_index[i] for i in sentence.split()]
    except:
        pass
    ############################
    
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_length_input,
                                                          padding='post')
    inputs = tf.convert_to_tensor(inputs)
    inference_batch_size = inputs.shape[0]
    result = ''

    enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
    enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)

    dec_h = enc_h
    dec_c = enc_c

    start_tokens = tf.fill([inference_batch_size], targ_lang.word_index['<start>'])
    end_token = targ_lang.word_index['<end>']

  # From official documentation
  # NOTE If you are using the BeamSearchDecoder with a cell wrapped in AttentionWrapper, then you must ensure that:
  # The encoder output has been tiled to beam_width via tfa.seq2seq.tile_batch (NOT tf.tile).
  # The batch_size argument passed to the get_initial_state method of this wrapper is equal to true_batch_size * beam_width.
  # The initial state created with get_initial_state above contains a cell_state value containing properly tiled final state from the encoder.

    enc_out = tfa.seq2seq.tile_batch(enc_out, multiplier=beam_width)
    decoder.attention_mechanism.setup_memory(enc_out)
    print("beam_with * [batch_size, max_length_input, rnn_units] :  3 * [1, 16, 1024]] :", enc_out.shape)

  # set decoder_inital_state which is an AttentionWrapperState considering beam_width
    hidden_state = tfa.seq2seq.tile_batch([enc_h, enc_c], multiplier=beam_width)
    decoder_initial_state = decoder.rnn_cell.get_initial_state(batch_size=beam_width*inference_batch_size, dtype=tf.float32)
    decoder_initial_state = decoder_initial_state.clone(cell_state=hidden_state)

  # Instantiate BeamSearchDecoder
    decoder_instance = tfa.seq2seq.BeamSearchDecoder(decoder.rnn_cell,beam_width=beam_width, output_layer=decoder.fc)
    decoder_embedding_matrix = decoder.embedding.variables[0]

  # The BeamSearchDecoder object's call() function takes care of everything.
    outputs, final_state, sequence_lengths = decoder_instance(decoder_embedding_matrix, start_tokens=start_tokens, end_token=end_token, initial_state=decoder_initial_state)
  # outputs is tfa.seq2seq.FinalBeamSearchDecoderOutput object. 
  # The final beam predictions are stored in outputs.predicted_id
  # outputs.beam_search_decoder_output is a tfa.seq2seq.BeamSearchDecoderOutput object which keep tracks of beam_scores and parent_ids while performing a beam decoding step
  # final_state = tfa.seq2seq.BeamSearchDecoderState object.
  # Sequence Length = [inference_batch_size, beam_width] details the maximum length of the beams that are generated


  # outputs.predicted_id.shape = (inference_batch_size, time_step_outputs, beam_width)
  # outputs.beam_search_decoder_output.scores.shape = (inference_batch_size, time_step_outputs, beam_width)
  # Convert the shape of outputs and beam_scores to (inference_batch_size, beam_width, time_step_outputs)
    final_outputs = tf.transpose(outputs.predicted_ids, perm=(0,2,1))
    beam_scores = tf.transpose(outputs.beam_search_decoder_output.scores, perm=(0,2,1))

    return final_outputs.numpy(), beam_scores.numpy()

In [28]:
def beam_summary(sentence):
    result, beam_scores = beam_evaluate_sentence(sentence)
    print(result.shape, beam_scores.shape)
    for beam, score in zip(result, beam_scores):
        print(beam.shape, score.shape)
        output = targ_lang.sequences_to_texts(beam)
        output = [a[:a.index('<end>')] for a in output]
        beam_score = [a.sum() for a in score]
        print('Input: %s' % (sentence))
        for i in range(len(output)):
            print('{} Predicted summarization: {}  {}'.format(i+1, beam_score[i], output[i]))

In [29]:
for text,summary in zip(df['Text'][-5:],df['Summary'][-5:]):
    print(beam_summary(text),'Real Summary: ',summary,'\n')

beam_with * [batch_size, max_length_input, rnn_units] :  3 * [1, 16, 1024]] : (3, 284, 1024)
(1, 3, 5) (1, 3, 5)
(3, 5) (3, 5)
Input: تساعد الزيوت العطريه الاساسيه علاج تساقط الشعر  فقد تم استعمال زيت اللافندر بنجاح الاشخاص يعانون الصلع  يتم مزج الزيوت ببعضها  فمثلا يمكن مزج زيت زيت الزعتر زيت الروزماري  توجد ادله كافيه تثبت فعاليه الزيوت علاج تساقط الشعر  وينصح استعمالها بوضع قطرات الجلد الانتظار لمده  ساعه  للتاكد كانت تسبب الحساسيه 
1 Predicted summarization: -9.383207321166992  الزيوت المستخده في التدليك 
2 Predicted summarization: -14.324695587158203  الزيوت العطريه 
3 Predicted summarization: -15.911822319030762  استخدام الزيوت الطبيعيه 
None Real Summary:  الزيوت العطريه 

beam_with * [batch_size, max_length_input, rnn_units] :  3 * [1, 16, 1024]] : (3, 284, 1024)
(1, 3, 7) (1, 3, 7)
(3, 7) (3, 7)
Input: يجب التركيز تناول الاطعمه الغنيه بالفيتامينات المعادن المختلفه للحفاظ صحه الشعر  فالشعر الصحي يعتمد نوعيه الطعام المتناول  يحتاج الشعر الحديد  الزنك  حمض الفوليك 
1 Predicted su